# Association Rules for trading in Python

This notebook uses the concepts of personalization and Association Rules to mine for stock events which appear together.

Just as Milk, Peanut Butter, Jelly, and Bread frequently occur together, events in the stock market occur together in various combinations. By converting stock price from a continuous variable to a descrite variable (a 'factor' in R or a 'category' in python), we can treat these events like items in the apriori algorithm. 

We will get the stock data from Quandl. You can register for a free account to access a lot of data with a key. My key has been removed from this notebook so you will need to provide a your own key to execute the cells. 

Author: Kyle Jones, Solutions Architect

Date: December 13, 2019

In [1]:
!pip install Quandl

  Running setup.py bdist_wheel for inflection ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import quandl

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [36]:
quandl.ApiConfig.api_key = 'YOUR-KEY'
df = quandl.get('WIKI/BP', start_date='2016-01-01')

In [37]:
df.tail() #using the free version from Quandl means we don't get the most recent data.

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2018-03-21,39.05,40.300,39.000,40.13,7436566.0,0.0,1.0,39.05,40.300,39.000,40.13,7436566.0
2018-03-22,39.59,39.600,38.920,39.03,6213172.0,0.0,1.0,39.59,39.600,38.920,39.03,6213172.0
2018-03-23,39.26,39.635,39.091,39.12,5325811.0,0.0,1.0,39.26,39.635,39.091,39.12,5325811.0
2018-03-26,40.33,40.450,39.865,40.42,6791082.0,0.0,1.0,40.33,40.450,39.865,40.42,6791082.0
2018-03-27,40.52,40.570,39.810,39.97,4740280.0,0.0,1.0,40.52,40.570,39.810,39.97,4740280.0


In [38]:
cols = ['Open', 'High', 'Low', 'Close']

for i in cols:
    df['{}_MA'.format(i)] = df[i].rolling(window=20).mean()
    df['t0_{}'.format(i)] = ["t0_{}_H".format(i) if x > y  else "t0_{}_L".format(i) for (x,y) in zip(df['{}'.format(i)],df['{}_MA'.format(i)])]
    df['t1_{}'.format(i)] = df['t0_{}'.format(i)].shift(periods=-1)
    df['t1_{}'.format(i)] = df['t1_{}'.format(i)].str.replace('0','1')

In [39]:
keep = ['t0_Open', 't1_Open', 't0_High', 't1_High', 't0_Low', 't1_Low', 't0_Close', 't1_Close']

df_c = df[keep]

records = []
for i in range(0, len(df_c)):
    records.append([str(df_c.values[i,j]) for j in range(0, len(df_c.columns))])

In [42]:
pd.DataFrame(records).to_csv('bp_stock.csv', header=False, index=False)

In [41]:
len(records)

561